Importing Libraries

In [ ]:
#DataPreprocessing Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Machine Learning Model Libraries
from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV

Data Preprocessing and Basic EDA

In [ ]:
df_test= pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")           
df = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv")

In [ ]:
#Row_ID can be dropped as it wont affect target variable
df.drop(["row_id"],axis=1,inplace=True)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df["target"].value_counts()

Target Value counts looks good as all categories have nearby same count. We can conclude that dataset is balanced and move further.

In [ ]:
df.duplicated().sum()

In [ ]:
#We can drop Duplicated values since it wont affect much at end
df.drop_duplicates(keep='first',inplace=True)

In [ ]:
df["target"].value_counts()

Still Dataset seems to be balanced. We can proceed for further analysis.



In [ ]:
df.describe()

In [ ]:
df.isna().sum()

No NULL values 😇

In [ ]:
threshold = 0.8
corr = df.corr()

corr_pairs = (
    corr[abs(corr) > threshold][corr != 1.0]
).unstack().dropna().to_dict()

unique_corr_pairs = pd.DataFrame(
    list(
        set([(tuple(sorted(key)), corr_pairs[key]) for key in corr_pairs])
    ), columns=['pair', 'corr']
)

unique_corr_pairs

In [ ]:
#We can drop 3 features which have correlation above 0.8.
df.drop(['A3T0G3C4','A1T2G6C1','A4T0G1C5'],axis=1,inplace=True)

Encoding categorical variables and training models

In [ ]:
#Label Encoding
le = preprocessing.LabelEncoder()
df.target=le.fit_transform(df['target'])

In [ ]:
#Defining dependent and independent features
X=df.drop(["target"],axis=1)
y=df.target

In [ ]:
#Paramenters for ET 
et_params = {
    'n_jobs':-1,
    'n_estimators': 300,
    'max_depth': None,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': 'auto',
    'max_samples': None
}

In [ ]:
#Training bagging with Extra trees as base estimator
#Training bagging with Extra trees as base estimator
clf = BaggingClassifier(base_estimator=ExtraTreesClassifier( **et_params),
                         n_estimators=10, random_state=0).fit(X, y)

Creating submission file

In [ ]:
df_test.drop(['A3T0G3C4','A1T2G6C1','A4T0G1C5'],axis=1,inplace=True)


In [ ]:
fin_pred=pd.DataFrame(index=df_test['row_id'])
df_test.drop(['row_id'],axis=1,inplace=True)
pred=clf.predict(df_test)
predictions=le.inverse_transform(pred)
fin_pred['target']=predictions

In [ ]:
fin_pred.head()

In [ ]:
fin_pred.to_csv("submission.csv")